In [104]:
## TOPIC Modelling

In [105]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [106]:
# Import Dataset
df= pd.read_csv('/Users/vibhuverma/Desktop/CLASSWORK/BUSINESS PRACTICUM/Anti-Corruption/Data Provided/USAID_Project_Data/USAID_Anticorruption_Projects_Database.csv')
df = df.drop(['rfp_rftop','final_res','mind_eval','final_eval','audit','interim_rep','addl_docs','addl_info'],axis=1)

In [107]:
# Convert to list
data = df.proj_desc.values.tolist()

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['The five components of this program seek to take what already exists and '
 'improve upon it, implement it and find ways to ensure that the progress '
 'achieved is sustainable over the long term. The project will help the '
 'government of the Islamic Republic of Afghanistan strengthen its High Office '
 'of Oversight (HOO) - making it a strong, effective institution that is able '
 'to lead, monitor, coordinate and report on efforts to combat corruption '
 'across the country. The project will work and implement a program of '
 'institutional development and sustainability. It will also support the '
 'office in carrying out its priority responsibilities in asset registration '
 'and verification, complaints management and case tracking, and coordination '
 'and monitoring of anticorruption performance across other government '
 'agencies.']


In [108]:
#tokenize and clean up using gensim 

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))


In [109]:
#lemmatization

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['component program seek take already exist improve implement find way ensure progress achieve sustainable long term project will help government strengthen high office oversight hoo make strong effective institution able lead monitor coordinate report effort combat corruption country project will work implement program institutional development sustainability will also support office carry priority asset registration verification complaint management case tracking coordination monitoring anticorruption performance other government agency', 'overall goal program support rule law strategy specifically pillar improve justice sector reduce corruption pillar build leadership justice sector civil society support pillar contractor attempt develop capacity judiciary law school pillar contractor work raise public awareness encourage citizen resolve dispute formal justice sector program divide component capacity building judiciary capacity building court management system capacity building facu

In [110]:
## TFIDF


vectorizer = TfidfVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             ngram_range=(1,3)             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [111]:
#Checking the sparcity

# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  9.346462864981383 %


In [112]:
## BASE MODEL



# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)


In [113]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -11455.377931215236
Perplexity:  1866.1054411160621
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [114]:
## HYPERPARAMETER TUNING

# Define Search Param
search_params = {'n_components': np.arange(5,30).tolist(), 'learning_decay': [.3,.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'learning_decay': [0.3, 0.5, 0.7, 0.9],
                         'n_components': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                                          16, 17, 18, 19, 20, 21, 22, 23, 24,
                                          25, 26, 27, 28, 29]})

In [115]:
## BEST TOPIC MODEL


# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))



Best Model's Params:  {'learning_decay': 0.5, 'n_components': 5}
Best Log Likelihood Score:  -2452.7496561109447
Model Perplexity:  808.4791541402814


In [116]:
### DOMINANT TOPIC IN EACH DOCUMENT


# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics




,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.030000,0.890000,0.030000,0.030000,0.030000,1
Doc1,0.030000,0.770000,0.140000,0.030000,0.030000,1
Doc2,0.050000,0.810000,0.050000,0.050000,0.050000,1
Doc3,0.730000,0.050000,0.140000,0.040000,0.040000,0
Doc4,0.040000,0.860000,0.040000,0.040000,0.040000,1
Doc5,0.030000,0.050000,0.850000,0.030000,0.030000,2
Doc6,0.030000,0.770000,0.140000,0.030000,0.030000,1
Doc7,0.080000,0.040000,0.800000,0.040000,0.040000,2
Doc8,0.040000,0.040000,0.860000,0.040000,0.040000,2
Doc9,0.030000,0.030000,0.870000,0.030000,0.040000,2


In [117]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
1      150.222153  -72.269554       1        1  45.690430
2       51.163364 -159.968018       2        1  39.581779
4      -73.144554   21.523815       3        1   7.137155
0      -66.410133 -110.606438       4        1   4.181114
3       54.417324   12.443444       5        1   3.409523, topic_info=                  Term       Freq      Total Category  logprob  loglift
25          assessment   5.000000   5.000000  Default  30.0000  30.0000
72          corruption  16.000000  16.000000  Default  29.0000  29.0000
222          political   6.000000   6.000000  Default  28.0000  28.0000
164         investment   4.000000   4.000000  Default  27.0000  27.0000
180              local  16.000000  16.000000  Default  26.0000  26.0000
..                 ...        ...        ...      ...      ...      ...
45   capacity building   0.160135   2.620236   Topic5  -5.7802   0.5836
232       professional   0.160134   3.029684   Topic5  -5.7803   0.4384
305                tax   0.160134   4.012777   Topic5  -5.7803   0.1574
14              afghan   0.160129   3.313637   Topic5  -5.7803   0.3488
106             engage   0.160125   3.020800   Topic5  -5.7803   0.4413

[273 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
2         1  0.643160  accountability
2         2  0.257264  accountability
7         1  0.611560           actor
9         1  0.771759  administration
9         2  0.192940  administration
...     ...       ...             ...
319       1  0.411590            unit
319       2  0.411590            unit
321       2  0.697090           woman
322       1  0.481009            work
322       2  0.481009            work

[336 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 5, 1, 4])

In [118]:
### TOPIC KEYWORDS ###

# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

,ability,access,accountability,accountable,achieve,action,activity,actor,address,administration,...,training,transition,transparency,transparency accountability,transparent,unit,use,woman,work,year
Topic0,0.200157,0.200141,0.200471,0.200123,0.200145,0.200261,0.200238,0.300457,0.201160,0.200141,...,0.200244,0.200103,0.200120,0.200120,0.200121,0.200146,0.200168,0.600677,0.200386,0.202620
Topic1,1.986705,4.317363,5.866041,2.090614,2.766748,1.816027,4.581274,2.742507,2.967293,4.653919,...,4.922204,0.203790,7.692333,3.205432,1.982327,1.488595,3.317933,0.202256,4.281885,1.722377
Topic2,0.878025,1.846096,2.851651,2.394676,1.261094,0.739854,2.156466,0.456948,2.368943,0.979358,...,3.361545,2.874712,3.048028,1.706248,3.961566,0.860320,1.174053,3.841524,4.880449,1.037981
Topic3,0.200194,0.200174,0.200141,0.200153,0.200180,0.200178,0.200166,0.200306,0.200180,0.200176,...,0.200143,0.200128,0.200149,0.200149,0.200150,0.200181,0.200203,0.200173,0.200171,0.200190
Topic4,0.209966,0.206529,0.200542,0.200154,0.273950,0.200080,0.418871,0.271964,0.259459,0.201008,...,0.200533,0.200047,0.200542,0.200349,0.203090,0.200361,0.413512,0.383516,0.431599,0.200764


In [119]:
### TOP 15 KEYWORDS FOR EACH TOPIC ###

# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,assessment,identify,political,right,corruption,country,structure,project design,democracy,stakeholder,woman,design,afghan,long,prevent
Topic 1,corruption,project,justice,public,strengthen,judicial,program,improve,law,legal,government,support,increase,reform,capacity
Topic 2,local,governance,government,program,local government,development,service,municipality,community,support,municipal,project,democratic,economic,capacity
Topic 3,program work,finance,benefit,improve public,stakeholder,project provide,approach,result,primary,anticorruption,office,partnership,economy,association,education
Topic 4,investment,trade,health,financial,regulatory,business,result,sector,benefit,environment,policy,private,reform,technical assistance,regulation


## Sentiment Analysis on Proj Res

In [122]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from textblob import TextBlob

In [123]:
## Change the proj_res type to string
df['proj_res'] = df['proj_res'].astype(str)

#lower casing
df['proj_res'] = df['proj_res'].apply(lambda x: " ".join(x.lower() for x in x.split()))

## remove punctuation
df['proj_res'] = df['proj_res'].str.replace('[^ws]','')

#stop words
stop = stopwords.words('english')
df['proj_res'] = df['proj_res'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

#stemming
st = PorterStemmer()
df['proj_res'] = df['proj_res'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))


In [124]:
## Define a function which can be applied to calculate the score for the whole dataset

def senti(x):
    return TextBlob(x).sentiment  

df['senti_score'] = df['proj_res'].apply(senti)

df.senti_score.head()

0    (0.0, 0.0)
1    (0.0, 0.0)
2    (0.0, 0.0)
3    (0.0, 0.0)
4    (0.0, 0.0)
Name: senti_score, dtype: object